In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from IPython.core.display import display, HTML, clear_output
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
from scipy import optimize
from scipy.stats import sem

import os
import sys

import matplotlib.pyplot as plt
import matplotlib.ticker
from matplotlib.patches import Rectangle
from matplotlib.colors import ListedColormap, BoundaryNorm

from venn import venn, pseudovenn
import seaborn as sns

from Bio import SeqIO, SeqUtils

from addict import Dict

import math

In [4]:
sys.path.append("../")
import plotting as my_plot

sys.path.append("./mean_field_elect/")
import proteomics_functions as prot_fun

# Difficult to remove (includes high-risk)

In [9]:
df_master = pd.read_csv('./generated_tables/df_master_dda_with_mean_properties.csv')

In [10]:
prot_fun.get_name_matches(df_master, './lit_data/biophorum_clean.csv', 'Biophorum DTR')
prot_fun.get_name_matches(df_master, './lit_data/biophorum_high_risk_clean.csv', 'Biophorum high-risk')
prot_fun.get_name_matches(df_master, './lit_data/levy_2014_clean.csv', 'Levy 2014 - Association')
prot_fun.get_name_matches(df_master, './lit_data/zhang_table_2_clean.csv', 'Zhang 2016 - Pro A')
prot_fun.get_name_matches(df_master, './lit_data/levy_2016_clean.csv', 'Levy 2016 - Coelution')
prot_fun.get_name_matches(df_master, './lit_data/molden_clean.csv', 'Molden 2021 - 29 DS')
prot_fun.get_name_matches(df_master, './lit_data/falkenberg_table_4_clean.csv', 'Falkenberg 2019 - 6 DS')
prot_fun.get_name_matches(df_master, './lit_data/kreimer_table_1_clean.csv', 'Kreimer 2017 - 1 DS')
prot_fun.get_name_matches(df_master, './lit_data/aboulaich_table_1_clean.csv', 'Aboulaich 2014 - Association')

In [14]:
papers = ['Biophorum DTR', 'Biophorum high-risk', 'Levy 2014 - Association', 'Zhang 2016 - Pro A', 
          'Levy 2016 - Coelution', 'Molden 2021 - 29 DS', 'Falkenberg 2019 - 6 DS', 'Kreimer 2017 - 1 DS',
          'Aboulaich 2014 - Association']

df_dtr = df_master[(df_master[papers[0]].notnull()) |
                   (df_master[papers[1]].notnull()) |
                   (df_master[papers[2]].notnull()) |
                   (df_master[papers[3]].notnull()) |
                   (df_master[papers[4]].notnull()) |
                   (df_master[papers[5]].notnull()) |
                   (df_master[papers[6]].notnull()) |
                   (df_master[papers[7]].notnull()) |
                   (df_master[papers[8]].notnull()) ]

df_dtr.reset_index(inplace=True, drop=True)

In [17]:
df_dtr.sort_values(by=['perfect_match', 'contains_actin', 'contains_ubiquitin'], inplace=True)

df_dtr = df_dtr[['accn', 'name', 'cq_g12', 'cq_eluate', 'hq_g12', 'hq_eluate', 'xq_g12',
                   'xq_eluate', 'pI_mean', 'mass_mean', 'net_charge_mean',
                   'net_charge_neg_mean', 'net_charge_pos_mean', 'charge_dens_C_m2_mean',
                   'charge_dens_neg_C_m2_mean', 'charge_dens_pos_C_m2_mean',
                   'cysteine_cont_percent_mean', 'cysteine_num_mean', 'desc_lower',
                   'perfect_match', 'contains_actin', 'contains_ubiquitin',
                   'desc_lower_2', 'Biophorum DTR', 'Biophorum high-risk',
                   'Levy 2014 - Association', 'Zhang 2016 - Pro A',
                   'Levy 2016 - Coelution', 'Molden 2021 - 29 DS',
                   'Falkenberg 2019 - 6 DS', 'Kreimer 2017 - 1 DS',
                   'Aboulaich 2014 - Association']]

In [20]:
df_dtr.to_csv('./generated_tables/difficult_to_remove_unreviewed.csv', index=False)

# Add impact to high-risk

In [30]:
df_dtr = pd.read_csv('./generated_tables/difficult_to_remove_reviewed.csv')
df_risk = df_dtr[df_dtr['Biophorum high-risk'].notnull()].copy()

path = './lit_data/biophorum_high_risk_clean.csv'
df_lit = pd.read_csv(path)

for i, cont in df_risk.iterrows():
    for col in ['impact', 'impact_type', 'function']:
        data = df_lit.loc[df_lit.desc_lower_2 == cont['Biophorum high-risk'], col].iloc[0] 
        df_risk.at[i, col] = data
        
df_risk.to_csv('./generated_tables/high_risk.csv', index=False)